# Sudoku miniproject

Below you will find the most elegant solution of sudoku solver coded by Peter Norvig. It's a good notation and implementation of two (simple) techniques that are enough to solve sudoku in a reasonable time.

You can find the original post here: http://norvig.com/sudoku.html

The best website about sudoku: http://www.sudokudragon.com/sudokutheory.htm

Using notations, I created the another sudoku solver solution.

### Sudoku Notation and Preliminary Notions

First we have to agree on some notation. A Sudoku puzzle is a grid of 81 squares; the majority of enthusiasts label the columns 1-9, the rows A-I, and call a collection of nine squares (column, row, or box) a unit and the squares that share a unit the peers. A puzzle leaves some squares blank and fills others with digits, and the whole idea is:
A puzzle is solved if the squares in each unit are filled with a permutation of the digits 1 to 9.
That is, no digit can appear twice in a unit, and every digit must appear once. This implies that each square must have a different value from any of its peers. Here are the names of the squares, a typical puzzle, and the solution to the puzzle:

```
 A1 A2 A3| A4 A5 A6| A7 A8 A9    4 . . |. . . |8 . 5     4 1 7 |3 6 9 |8 2 5 
 B1 B2 B3| B4 B5 B6| B7 B8 B9    . 3 . |. . . |. . .     6 3 2 |1 5 8 |9 4 7
 C1 C2 C3| C4 C5 C6| C7 C8 C9    . . . |7 . . |. . .     9 5 8 |7 2 4 |3 1 6 
---------+---------+---------    ------+------+------    ------+------+------
 D1 D2 D3| D4 D5 D6| D7 D8 D9    . 2 . |. . . |. 6 .     8 2 5 |4 3 7 |1 6 9 
 E1 E2 E3| E4 E5 E6| E7 E8 E9    . . . |. 8 . |4 . .     7 9 1 |5 8 6 |4 3 2 
 F1 F2 F3| F4 F5 F6| F7 F8 F9    . . . |. 1 . |. . .     3 4 6 |9 1 2 |7 5 8 
---------+---------+---------    ------+------+------    ------+------+------
 G1 G2 G3| G4 G5 G6| G7 G8 G9    . . . |6 . 3 |. 7 .     2 8 9 |6 4 3 |5 7 1 
 H1 H2 H3| H4 H5 H6| H7 H8 H9    5 . . |2 . . |. . .     5 7 3 |2 9 1 |6 8 4 
 I1 I2 I3| I4 I5 I6| I7 I8 I9    1 . 4 |. . . |. . .     1 6 4 |8 7 5 |2 9 3 
```

Every square has exactly 3 units and 20 peers. For example, here are the units and peers for the square C2:

```
    A2   |         |                    |         |            A1 A2 A3|         |         
    B2   |         |                    |         |            B1 B2 B3|         |         
    C2   |         |            C1 C2 C3| C4 C5 C6| C7 C8 C9   C1 C2 C3|         |         
---------+---------+---------  ---------+---------+---------  ---------+---------+---------
    D2   |         |                    |         |                    |         |         
    E2   |         |                    |         |                    |         |         
    F2   |         |                    |         |                    |         |         
---------+---------+---------  ---------+---------+---------  ---------+---------+---------
    G2   |         |                    |         |                    |         |         
    H2   |         |                    |         |                    |         |         
    I2   |         |                    |         |                    |         |    
```

In [1]:
#notation
#create squares, units and peers

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]

digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
squares  = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
units = dict((s, [u for u in unitlist if s in u]) 
             for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in squares)

print(units['A1'])


[['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'], ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'], ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]


### Sudoku grid
Now that we have squares, units, and peers, the next step is to define the Sudoku playing grid. Actually we need two representations: First, a textual format used to specify the initial state of a puzzle; we will reserve the name grid for this. Second, an internal representation of any state of a puzzle, partially solved or complete; this we will call a values collection because it will give all the remaining possible values for each square. For the textual format (grid) we'll allow a string of characters with 1-9 indicating a digit, and a 0 or period specifying an empty square. All other characters are ignored (including spaces, newlines, dashes, and bars). So each of the following three grid strings represent the same puzzle:


In [2]:
# parser

def grid_values(grid):
    "Convert grid into a dict of {square: char} with '0' or '.' for empties."
    chars = [c for c in grid if c in digits or c in '0.']
    assert len(chars) == 81
    return dict(zip(squares, chars))

def display(values):
    "Display these values as a 2-D grid."
    width = 1+max(len(values[s]) for s in squares)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '') for c in cols))
        if r in 'CF': print(line)
    print()

# sample sudoku
grid1 = '003020600900305001001806400008102900700000008006708200002609500800203009005010300'
grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
grid3 = "85...24..72......9..4.........1.7..23.5...9...4...........8..7..17..........36.4."

print('Easy sudoku')
display(grid_values(grid1))
print('Medium sudoku')
display(grid_values(grid2))
print('Hard sudoku')
display(grid_values(grid3))

Easy sudoku
0 0 3 |0 2 0 |6 0 0 
9 0 0 |3 0 5 |0 0 1 
0 0 1 |8 0 6 |4 0 0 
------+------+------
0 0 8 |1 0 2 |9 0 0 
7 0 0 |0 0 0 |0 0 8 
0 0 6 |7 0 8 |2 0 0 
------+------+------
0 0 2 |6 0 9 |5 0 0 
8 0 0 |2 0 3 |0 0 9 
0 0 5 |0 1 0 |3 0 0 

Medium sudoku
4 . . |. . . |8 . 5 
. 3 . |. . . |. . . 
. . . |7 . . |. . . 
------+------+------
. 2 . |. . . |. 6 . 
. . . |. 8 . |4 . . 
. . . |. 1 . |. . . 
------+------+------
. . . |6 . 3 |. 7 . 
5 . . |2 . . |. . . 
1 . 4 |. . . |. . . 

Hard sudoku
8 5 . |. . 2 |4 . . 
7 2 . |. . . |. . 9 
. . 4 |. . . |. . . 
------+------+------
. . . |1 . 7 |. . 2 
3 . 5 |. . . |9 . . 
. 4 . |. . . |. . . 
------+------+------
. . . |. 8 . |. 7 . 
. 1 7 |. . . |. . . 
. . . |. 3 6 |. 4 . 



### Parser
Now for values. One might think that a 9 x 9 array would be the obvious data structure. But squares have names like 'A1', not (0,0). Therefore, values will be a dict with squares as keys. The value of each key will be the possible digits for that square: a single digit if it was given as part of the puzzle definition or if we have figured out what it must be, and a collection of several digits if we are still uncertain. This collection of digits could be represented by a Python set or list, but I chose instead to use a string of digits (we'll see why later). So a grid where A1 is 7 and C7 is empty would be represented as {'A1': '7', 'C7': '123456789', ...}.

In [3]:
def parse_grid(grid):
    start = grid_values(grid)
    values = dict((s,digits) for s in squares)
    for s, d in start.items():
        if d in digits:
            values[s] = d
    return values

sudoku_values = (grid_values(grid1))
                 
display(sudoku_values)

0 0 3 |0 2 0 |6 0 0 
9 0 0 |3 0 5 |0 0 1 
0 0 1 |8 0 6 |4 0 0 
------+------+------
0 0 8 |1 0 2 |9 0 0 
7 0 0 |0 0 0 |0 0 8 
0 0 6 |7 0 8 |2 0 0 
------+------+------
0 0 2 |6 0 9 |5 0 0 
8 0 0 |2 0 3 |0 0 9 
0 0 5 |0 1 0 |3 0 0 



# <span style="color:red"> Constraint Propagation - refactored code </span>

Sudoku  is a  number-placement puzzle. Sudoku is  a 9×9 grid and it contains nine 3x3 smaller grid. Sudoku is based on filled puzzle in such way that each column, each row, and each of the nine 3×3 subgrids contain  all of the digits from 1 to 9.

The most important principle of sudoku is that  the same single integer may not appear twice in the same row, column, or any of the nine 3×3 subregions of the 9×9 playing board.

eliminate() and only_choice() functions are respectively a 1 and a 2 strategy:
(1) If a square has only one possible value, then eliminate that value from the square's peers. 

(2) If a unit has only one possible place for a value, then put the value there.

reduce_puzzle() function solves sudoku using eliminate() and only_choice() functions.


In [4]:
def eliminate(values):
    """
    Iterate through all squares and every time 
       if there is a square with one value, 
       then eliminate this value from the peers

    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """

    for s in squares:
        d = values[s]
        if len(d) == 1:
            for p in peers[s]:
                if values[p].find(d) != -1:
                    values[p] = values[p].replace(d,'')
    return values
    
def only_choice(values):
    """
    Iterate through all squares and every time
        if there is a square with a value that only fits in one square, 
        assign the value to this square

    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    
    for s in squares:
        if len(values[s]) > 1:
            for u in units[s]:
                for c in u:
                    for d in values[c]:
                        if not any (p for p in peers[s]):
                            if d in values[p]:
                                values[s] = d
                                break
    return values

def reduce_puzzle(values):
    """
    Solve sudoku using eliminate() and only_choice()
    
    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    previous = dict.fromkeys(values.keys())
    while previous != values:
        previous = dict(values)
        eliminate(values)
        only_choice(values)
            
    return values


Function solve() solves sudoku using reduce_puzzle() function.

In [5]:
def my_solve(values):
    """
    Solve a sudoku using constraint propagation, sub-group exclusion and hidden twins techniques
    
    input: A sudoku in dictionary form.
    output: The resulting sudoku in dictionary form.
    """
        
    reduce_puzzle(values)
                         
    return values

sudoku = grid_values(grid1)
print('Sudoku to solve')
display(sudoku)

sudoku_values = parse_grid(grid1)
my_solve(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)

sudoku = grid_values(grid2)
print('Sudoku to solve')
display(sudoku)

sudoku_values = parse_grid(grid2)
my_solve(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)

sudoku = grid_values(grid3)
print('Sudoku to solve')
display(sudoku)

sudoku_values = parse_grid(grid3)
my_solve(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)

Sudoku to solve
0 0 3 |0 2 0 |6 0 0 
9 0 0 |3 0 5 |0 0 1 
0 0 1 |8 0 6 |4 0 0 
------+------+------
0 0 8 |1 0 2 |9 0 0 
7 0 0 |0 0 0 |0 0 8 
0 0 6 |7 0 8 |2 0 0 
------+------+------
0 0 2 |6 0 9 |5 0 0 
8 0 0 |2 0 3 |0 0 9 
0 0 5 |0 1 0 |3 0 0 

Solved Sudoku
4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 

Sudoku to solve
4 . . |. . . |8 . 5 
. 3 . |. . . |. . . 
. . . |7 . . |. . . 
------+------+------
. 2 . |. . . |. 6 . 
. . . |. 8 . |4 . . 
. . . |. 1 . |. . . 
------+------+------
. . . |6 . 3 |. 7 . 
5 . . |2 . . |. . . 
1 . 4 |. . . |. . . 

Solved Sudoku
   4      1679   12679  |  139     2369    1269  |   8      1239     5    
 26789     3    1256789 | 14589   24569  1245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569 1245689 | 12369   12349   123469 
-------------------

As you can see, strategy 1 and 2 are simple to being implemented, but they can solve only simple sudoku's grids. Only grid1 was solved. 

In http://www.sudokudragon.com/sudokustrategy.htm you can read about different strategies, such as naked twins or hidden twins.

From sudoku dragon:
You may find you need to use the twin (or triplet) exclusion rules in order to solve more challenging Sudoku puzzles. It is the strategy to use when simpler strategies have been applied and you are still stuck. In essence it is all about spotting matching patterns of possibilities in a group (row, column or region). Spotting these groups takes time and it is difficult to keep track of these in your head, so this is where you need pencil and paper (or the Sudoku Dragon puzzle solver).

If you have two or more unallocated squares in a region and there are two numbers that can only go in the same two squares and no others in that group you have a twin. This does not directly help to allocate squares as the number could go in either of them. However, if the two squares have another possible number then this number can be safely eliminated as an option. It is excluded because of the presence of the hidden twin in the group. 

Another way to exclude possibilities in a group is with naked twins. In this case the twin squares are evident on their own (and so they are termed 'naked' to distinguish them from the previous 'hidden' case) and these are used to exclude possibilities in other squares in the same group. 

http://www.sudokudragon.com/guidenakedtwins.htm

### Function: naked_twins() and hidden_twins():

In [6]:
def naked_twins(values):
    for s,d in squares:
        if len(d) == 2:
            for u in units[s]:
                for c in u:
                    if values[c] == d and c !=s:
                        for c2 in u:
                            values[c2] = values[c2].replace(d[0], '')
                            values[c2] = values[c2].replace(d[1], '')
                        values[s] = d
                        values[c] = d
    return values

def hidden_twins(values):
    for s, d in squares:
        if len(d) > 1:
            for u in units[s]:
                for c1 in u:
                    if len(values[c1]) > 1 and c1 !=s:
                        for first in d:
                            if values[c1].find(first) != -1:
                                for second in d:
                                    if values[c1].find(second) != -1 and second != first:
                                        first_match = first
                                        second_match = second
                            else:
                                first_match = 0
                                second_match = 0
                        if first_match != 0:
                            for c2 in u:
                                if c2 !=s and first_match in values[c2] and second_match in values[c2]:
                                    values[s] = first_match + second_match
                                    values[c2] = first_match + second_match                
    return values

### Fuction solve_naked() using naked_twins() strategies and function solve_hidden() using hidden_twins() strategies:

In [7]:
def solve_naked(values):
    """
    Solve a sudoku using constraint propagation, sub-group exclusion and hidden twins techniques
    
    input: A sudoku in dictionary form.
    output: The resulting sudoku in dictionary form.
    """
        
    previous = dict.fromkeys(values.keys())
    while previous != values:
        previous = dict(values)
        values = eliminate(values)
        values = naked_twins(values)
        values = only_choice(values)
                         
    return values

def solve_hidden(values):
    """
    Solve a sudoku using constraint propagation, sub-group exclusion and hidden twins techniques
    
    input: A sudoku in dictionary form.
    output: The resulting sudoku in dictionary form.
    """
        
    previous = dict.fromkeys(values.keys())
    while previous != values:
        previous = dict(values)
        values = eliminate(values)
        values = hidden_twins(values)
        values = only_choice(values)
                         
    return values

Solving some grids using my_solve(), solve_naked() and solve_hidden()  functions. Check, that implemented method can solve sudoku puzzle. 

Sample grids (from http://norvig.com/sudoku.html):

In [8]:
import time

grid1 = "003020600900305001001806400008102900700000008006708200002609500800203009005010300"
grid2 = "003020600900305001001806400008102900700000008006708200002609500800203009005010300"
grid3 = "480006902002008001900370060840010200003704100001060049020085007700900600609200018"
grid4 = "050807020600010090702540006070020301504000908103080070900076205060090003080103040"
grid5 = "500400060009000800640020000000001008208000501700500000000090084003000600060003002"

sudoku = grid_values(grid1)
print('Sudoku to solve')
display(sudoku)
print('my_solve()')

sudoku_values = parse_grid(grid1)
start_time = time.time()
my_solve(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)
print("Solved: %s seconds" % (time.time() - start_time))        

print('solve_naked()')

sudoku_values = parse_grid(grid1)
solve_naked(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)
print("Solved: %s seconds" % (time.time() - start_time))        

print('solve_hidden()')

sudoku_values = parse_grid(grid1)
start_time = time.time()
solve_hidden(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)
print("Solved: %s seconds" % (time.time() - start_time))        

sudoku = grid_values(grid2)
print('Sudoku to solve')
display(sudoku)
print('my_solve()')

sudoku_values = parse_grid(grid2)
start_time = time.time()
my_solve(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)
print("Solved: %s seconds" % (time.time() - start_time))        

print('solve_naked()')

sudoku_values = parse_grid(grid2)
start_time = time.time()
solve_naked(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)
print("Solved: %s seconds" % (time.time() - start_time))        

print('solve_hidden()')

sudoku_values = parse_grid(grid2)
start_time = time.time()
solve_hidden(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)
print("Solved: %s seconds" % (time.time() - start_time))        

sudoku = grid_values(grid3)
print('Sudoku to solve')
display(sudoku)
print('my_solve()')

sudoku_values = parse_grid(grid3)
start_time = time.time()
my_solve(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)
print("Solved: %s seconds" % (time.time() - start_time))        

print('solve_naked()')

sudoku_values = parse_grid(grid3)
start_time = time.time()
solve_naked(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)
print("Solved: %s seconds" % (time.time() - start_time))        

print('solve_hidden()')

sudoku_values = parse_grid(grid3)
start_time = time.time()
solve_hidden(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)
print("Solved: %s seconds" % (time.time() - start_time))        

sudoku = grid_values(grid4)
print('Sudoku to solve')
display(sudoku)
print('my_solve()')

sudoku_values = parse_grid(grid4)
start_time = time.time()
my_solve(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)
print("Solved: %s seconds" % (time.time() - start_time))        

print('solve_naked()')

sudoku_values = parse_grid(grid4)
start_time = time.time()
solve_naked(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)
print("Solved: %s seconds" % (time.time() - start_time))        

print('solve_hidden()')

sudoku_values = parse_grid(grid4)
start_time = time.time()
solve_hidden(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)
print("Solved: %s seconds" % (time.time() - start_time))        

sudoku = grid_values(grid5)
print('Sudoku to solve')
display(sudoku)
print('my_solve()')

sudoku_values = parse_grid(grid5)
start_time = time.time()
my_solve(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)
print("Solved: %s seconds" % (time.time() - start_time))        

print('solve_naked()')

sudoku_values = parse_grid(grid5)
start_time = time.time()
solve_naked(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)
print("Solved: %s seconds" % (time.time() - start_time))        

print('solve_hidden()')

sudoku_values = parse_grid(grid5)
start_time = time.time()
solve_hidden(sudoku_values)
print('Solved Sudoku')
display(sudoku_values)
print("Solved: %s seconds" % (time.time() - start_time))        


Sudoku to solve
0 0 3 |0 2 0 |6 0 0 
9 0 0 |3 0 5 |0 0 1 
0 0 1 |8 0 6 |4 0 0 
------+------+------
0 0 8 |1 0 2 |9 0 0 
7 0 0 |0 0 0 |0 0 8 
0 0 6 |7 0 8 |2 0 0 
------+------+------
0 0 2 |6 0 9 |5 0 0 
8 0 0 |2 0 3 |0 0 9 
0 0 5 |0 1 0 |3 0 0 

my_solve()
Solved Sudoku
4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 

Solved: 0.019271135330200195 seconds
solve_naked()
Solved Sudoku
4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 

Solved: 0.0297548770904541 seconds
solve_hidden()
Solved Sudoku
4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7

### Conclusions

In my mind,  the eliminate () and only_choice () methods are able to solve a large part of the easy sudoku puzzles. However, the strategy of naked or hidden twins supports the above-mentioned methods, but not be enough to solving more difficult sudoku puzzles. 

I suggest thinking about implementing other strategies, such as subgroup exclusion role, general permutation role, X-Wing and Swordfish.

Reading http://norvig.com/sudoku.html, the way proposed by Norvig is better than method implemented by me. 

### Laboratory 2 - 13.05.2019

Adding time to methods:


In [22]:
def howTime(method, grid):
    start = time.time()
    if method == 1:
        my_solve(grid)
    elif method == 2:
        solve_naked(grid)
    else:
        solve_hidden(grid)
    finish = time.time()
    t = finish - start
    return t 

sudoku = grid_values(grid5)
sudoku_values = parse_grid(grid5)
print('My_solvE() Time:')
print(howTime(1, sudoku_values))
print('solve_naked() Time:')
print(howTime(2, sudoku_values))
print('solve_hidden() Time:')
print(howTime(3, sudoku_values))

My_solvE() Time
0.01714611053466797
solve_naked() Time
0.002843141555786133
solve_hidden() Time
0.0029277801513671875


### TODO:
- generator sudoku
- search function implementations (based on Norvig)

### Generation sudoku

In [23]:
import random

def generation_grid():
    
    grid = dict((s,digits) for s in squares)
    while my_dygits > 7:
        for i in range(0, 17):
            c = random.choice(squares)
            if len(c) != 1:
                n = random.choice(digits)
                for u in units[c]:
                    if u == c or n == grid(u):
                        n = random.choice(digits)
                    else:
                        grid(c) = n
        my_digits = [grid[s] for s in squares if len(grid[s]) == 1] 

    
    
    return grid

SyntaxError: can't assign to function call (<ipython-input-23-d7dc445a5473>, line 15)

### Search function